**Copy this function everywhere you need to handle missing weeks**

In [4]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
year_start=2014
year_end=2022

In [16]:
def get_weeks(year_start,year_end):
    # The date range that we query
    datelist = pd.date_range(f'{year_start}-01-01', f'{year_end+1}-01-01').to_series()

    # Extract year and week 
    year = datelist.apply(lambda x: x.strftime('%Y')) 
    week = datelist.apply(lambda x: x.strftime('%U')).astype(int)

    # Combine years and their weeks in one table
    data_years = pd.DataFrame(data=[year, week], index=['year', 'week']).T

    # Get number of weeks per year
    data_weeks = data_years.groupby(['year']).max().to_dict('index')

    # Get a list of weeks per year
    week_ranges=[]
    for year in data_weeks.keys():
        week_ranges.append(list(range(1,data_weeks[year]['week']+1)))

    # Create a dict with year and exact week numbers
    weeks_per_year = dict(zip(list(data_weeks.keys()), week_ranges))
    
    return weeks_per_year

In [19]:
def find_missing_weeks(data):
    year_start = data.year.min()
    year_end = data.year.max()
    
    weeks_per_year=get_weeks(year_start,year_end)
    actual_data = data.groupby('year')['week_number'].apply(set).apply(list)

    weeks_to_add={}
    for year in range(year_start,year_end+1):
        weeks1 = get_weeks(year_start,year_end)[str(year)]
        weeks2 = actual_data[year]
        weeks_to_add[year] = (list(set(weeks1) - set(weeks2)))
        
    return weeks_to_add

In [20]:
def handle_missing_weeks(data):
    weeks_to_add = find_missing_weeks(data)
    
    for year in weeks_to_add.keys():
        for week in weeks_to_add[year]:
            data = data.append({'year':year, 'week_number':week, 'content':0}, ignore_index=True)
            
    return data.sort_values(['year', 'week_number']).reset_index(drop=True)

In [18]:
# get_weeks(year_start,year_end)